# Step 1: Build a forecasting model

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [ ]:
# read dataset
df=pd.read_csv('Advertising Budget and Sales.csv')
df = df.drop(df.columns[0], axis=1)

In [ ]:
# add (fake) dates in prophet formatting
n_days = df.shape[0]
df['ds'] = pd.date_range(start="2022-01-01", periods=n_days, freq="D")
df['y'] = df['Sales ($)']

In [ ]:
# create in an out of time testing datasets
print(df.shape[0])
df_prophet_train = df[df['ds'] < pd.to_datetime('2022-06-01')]
df_prophet_test = df[df['ds'] >= pd.to_datetime('2022-06-01')]
df_prophet_test2 = df_prophet_test.copy()
print(df_prophet_train.shape[0]+df_prophet_test.shape[0])

In [ ]:
# initialize and fit prophet model
model = Prophet(yearly_seasonality=True, daily_seasonality=True)
model.add_regressor('TV Ad Budget ($)')
model.add_regressor('Radio Ad Budget ($)')
model.add_regressor('Newspaper Ad Budget ($)')
model.fit(df_prophet_train)

In [ ]:
# use forecasting model for the out of time test sample
forecast = model.predict(df_prophet_test)

In [ ]:
# merge actuals and predicteds to assess performance
result_df = pd.merge(df, forecast[['ds', 'yhat']], on='ds', how='left')
result_df['prediction'] = np.where(result_df['yhat'].isnull(), result_df['y'], result_df['yhat'])

In [ ]:
# Plot the data
plt.figure(figsize=(10, 6))  # Set figure size
plt.plot(result_df["ds"], result_df["y"], label="Sales", marker="o", color='k')
plt.plot(result_df["ds"], result_df["yhat"], label="Predicted Sales", marker="s", color='b')

# Add a vertical line for 2024-01-01
highlight_date = pd.Timestamp("2022-06-01")
plt.axvline(x=highlight_date, color="red", linestyle="--", label="2024-01-01")

# Rotate x-axis labels
plt.xticks(rotation=90)

# Add labels, title, and legend
plt.xlabel("Date")
plt.ylabel("Values")
plt.title("Line Plot with Rotated Dates")
plt.legend()

# Show the plot
plt.tight_layout()  # Adjust layout to prevent cutoff of labels
plt.show()

In [ ]:
print(result_df[result_df['yhat'].notnull()]['y'].sum())
print(result_df[result_df['yhat'].notnull()]['yhat'].sum())

# Step 2: Use model for "what if" scenarios

Using the model, we wish to find the optimal spend across the channels to maximise the number of leads.

In [ ]:
available_spend = df_prophet_test['TV Ad Budget ($)'].sum() + df_prophet_test['Radio Ad Budget ($)'].sum() + df_prophet_test['Newspaper Ad Budget ($)'].sum()

print(f"Available Spend: ${available_spend}")

In [ ]:
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

# set a marketing budget
budget=10000

def predict_leads(tv_spend, radio_spend, newspaper_spend, model, df_prophet_test):

    # determine the number of rows to be used for uniform distribution of spend*
    # uniform distribution of spend may not be the "optimal"
    nrows=df_prophet_test.shape[0]
    
    # Assign the ad budgets to the dataframe columns
    df_prophet_test['TV Ad Budget ($)'] = [tv_spend/nrows] * nrows 
    df_prophet_test['Radio Ad Budget ($)'] = [radio_spend/nrows] * nrows
    df_prophet_test['Newspaper Ad Budget ($)'] = [newspaper_spend/nrows] * nrows
    
    # Make the forecast using the model
    forecast = model.predict(df_prophet_test)
    
    # Return the sum of the predicted leads (you can use other metrics like the average or max)
    return forecast['yhat'].sum()  # Assuming 'yhat' is the predicted leads

# Define the search space for TV and Radio spend (the Newspaper spend will be calculated from the total budget)
space = [
    Real(0, budget, name='tv_spend'),  # From 0 to 100,000 for TV spend
    Real(0, budget, name='radio_spend')  # From 0 to 100,000 for Radio spend
]

# Load your trained Prophet model (this is a dummy load function, replace it with actual model loading)
prophet_model = model

# Objective function to minimize (negative leads because we want to maximize leads)
@use_named_args(space)
def objective(tv_spend, radio_spend):
    # Calculate the remaining budget for Newspaper spend (total budget is 100,000)
    newspaper_spend = budget - tv_spend - radio_spend
    
    # If the Newspaper spend is negative, return a very low (negative) value
    if newspaper_spend < 0:
        return 0  # This ensures the optimizer doesn't choose invalid combinations
    
    # Get the predicted leads from the Prophet model
    predicted_leads = predict_leads(tv_spend, radio_spend, newspaper_spend, prophet_model, df_prophet_test)
    
    return -predicted_leads  # Minimize the negative of the predicted leads to maximize it

# Run the Bayesian Optimization
result = gp_minimize(objective, space, n_calls=50, random_state=42)

# Output the best results
print("Best TV Spend: ", result.x[0])
print("Best Radio Spend: ", result.x[1])

# Calculate the optimal Newspaper Spend (since the total is budget)
best_newspaper_spend = budget - result.x[0] - result.x[1]
print("Best Newspaper Spend: ", best_newspaper_spend)

# Maximum Predicted Leads
print("Maximum Predicted Leads: ", -result.fun)  # Negate the value to get the actual predicted leads

# Step 3: Compare baseline with "optimal"

In [ ]:
# original
print(result_df['yhat'].sum())

best_view = pd.DataFrame()

tv_spend = 2272
radio_spend = 701
newspaper_spend = 26
nrows = df_prophet_test.shape[0]

# Assign the ad budgets to the dataframe columns
best_view['ds'] = df_prophet_test['ds']
best_view['TV Ad Budget ($)'] = [tv_spend/nrows] * nrows 
best_view['Radio Ad Budget ($)'] = [radio_spend/nrows] * nrows
best_view['Newspaper Ad Budget ($)'] = [newspaper_spend/nrows] * nrows
display(best_view)

# Make the forecast using the model
forecast = model.predict(best_view)

bestview_df = pd.merge(df, forecast[['ds', 'yhat']], on='ds', how='left')
bestview_df['prediction'] = np.where(bestview_df['yhat'].isnull(), bestview_df['y'], bestview_df['yhat'])
display(bestview_df)

# Plot the data
plt.figure(figsize=(10, 6))  # Set figure size
plt.plot(bestview_df["ds"], bestview_df["y"], label="Sales", marker="o", color='k')
plt.plot(bestview_df["ds"], bestview_df["yhat"], label="Predicted Sales", marker="s", color='b')

# Add a vertical line for 2024-01-01
highlight_date = pd.Timestamp("2022-06-01")
plt.axvline(x=highlight_date, color="red", linestyle="--", label="2024-01-01")

# Rotate x-axis labels
plt.xticks(rotation=90)

# Add labels, title, and legend
plt.xlabel("Date")
plt.ylabel("Values")
plt.title("Line Plot with Rotated Dates")
plt.legend()

# Show the plot
plt.tight_layout()  # Adjust layout to prevent cutoff of labels
plt.show()

In [ ]:
bestview_df.yhat.sum()